In [1]:
%pip install folium

  Using cached folium-0.14.0-py2.py3-none-any.whl (102 kB)
  Using cached branca-0.6.0-py3-none-any.whl (24 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install utm


  Using cached utm-0.7.0.tar.gz (8.7 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6085 sha256=4f0c470da7b158c2fae888b18c97a4748cf31d334fe81cf7d1eec5ccd605e079
  Stored in directory: /Users/me/Library/Caches/pip/wheels/45/32/60/17655df214d7e5eeea1f5f76a8f590d9849a903f24b377c290
Successfully built utm
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install geocoder

  Using cached geocoder-1.38.1-py2.py3-none-any.whl (98 kB)
  Using cached ratelim-0.1.6-py2.py3-none-any.whl (4.0 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install geopy

  Using cached geopy-2.3.0-py3-none-any.whl (119 kB)
  Using cached geographiclib-2.0-py3-none-any.whl (40 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import ssl
import pandas as pd
import numpy as np
import utm
import folium
from folium import plugins 
import ipywidgets
import geocoder
import geopy



ssl._create_default_https_context = ssl._create_unverified_context

censo_actividad = pd.read_csv("censo_actividad_2023.csv",sep=";",low_memory=False)
pd.set_option('display.max_columns',None)


FileNotFoundError: [Errno 2] No such file or directory: 'censo_actividad_2023.csv'

In [5]:
censo_actividad.head()
df = censo_actividad[["id_local","desc_distrito_local","id_barrio_local","desc_barrio_local",\
    "cod_barrio_local","coordenada_x_local","coordenada_y_local","desc_situacion_local",\
        "rotulo","desc_seccion","id_division","desc_division","desc_epigrafe"]].copy()
nf = df.reindex(columns=["rotulo","desc_epigrafe","desc_seccion","desc_division","id_division",\
    "desc_distrito_local","desc_barrio_local","id_barrio_local","cod_barrio_local","desc_situacion_local",\
        "coordenada_x_local","coordenada_y_local","id_local"]).copy()
mf = nf[nf.id_division==str(56)].copy()

mf



,rotulo,desc_epigrafe,desc_seccion,desc_division,id_division,desc_distrito_local,desc_barrio_local,id_barrio_local,cod_barrio_local,desc_situacion_local,coordenada_x_local,coordenada_y_local,id_local
1,CASA RUA,BAR CON COCINA,HOSTELERÍA,SERVICIOS DE COMIDAS Y BEBIDAS,56,CENTRO,SOL,106,6,Abierto,439909.60,4474131.5,10000455
2,VIANDAS DE SALAMANCA,BAR SIN COCINA,HOSTELERÍA,SERVICIOS DE COMIDAS Y BEBIDAS,56,CENTRO,SOL,106,6,Abierto,440195.60,4474478.5,10000463
4,SIBUYA,BAR RESTAURANTE,HOSTELERÍA,SERVICIOS DE COMIDAS Y BEBIDAS,56,CENTRO,JUSTICIA,104,4,Abierto,441036.60,4475026.5,10000473
5,LA TAURINA,BAR RESTAURANTE,HOSTELERÍA,SERVICIOS DE COMIDAS Y BEBIDAS,56,CENTRO,SOL,106,6,Abierto,440477.60,4474250.5,10000530
6,ESCUELA INFANTIL LA PALOMA,SERVICIOS DE COMEDOR EN CENTROS EDUCATIVOS Y C...,HOSTELERÍA,SERVICIOS DE COMIDAS Y BEBIDAS,56,CENTRO,PALACIO,101,1,Abierto,439620.60,4473597.5,10000684
...,...,...,...,...,...,...,...,...,...,...,...,...,...
168137,E.I. LOS PEQUES,SERVICIOS DE COMEDOR EN CENTROS EDUCATIVOS Y C...,HOSTELERÍA,SERVICIOS DE COMIDAS Y BEBIDAS,56,VILLAVERDE,SAN ANDRES,1701,1,Abierto,439535.53,4466202.5,285028672
168290,JARAGUA SPORT,BAR CON COCINA,HOSTELERÍA,SERVICIOS DE COMIDAS Y BEBIDAS,56,CIUDAD LINEAL,PUEBLO NUEVO,1502,2,Abierto,445754.56,4475408.5,285044646
168302,MADRID 2012,BAR CON COCINA,HOSTELERÍA,SERVICIOS DE COMIDAS Y BEBIDAS,56,CIUDAD LINEAL,PUEBLO NUEVO,1502,2,Cerrado,445557.56,4475250.5,285044701
168489,SIN ROTULO,CAFETERIA,HOSTELERÍA,SERVICIOS DE COMIDAS Y BEBIDAS,56,CIUDAD LINEAL,PUEBLO NUEVO,1502,2,Cerrado,446457.56,4475341.5,285045103


In [6]:
mf = mf.astype({"rotulo":"string","desc_epigrafe":"string","desc_seccion":"string",\
                "desc_division":"string","id_division":"int","desc_distrito_local":"string",\
                "desc_situacion_local":"string","desc_barrio_local":"string"})
fl = mf[mf["desc_distrito_local"].str.contains("SALAMANCA")|mf["desc_distrito_local"].str.contains("CHAMBERI")]
fl= fl.sort_values(["desc_distrito_local"],ascending=True).copy()
xs = fl[fl["coordenada_x_local"]>0]
xz = xs[xs["coordenada_y_local"]>0]
clean = xz[xz["desc_situacion_local"].str.contains("Abierto")]
plis = clean[clean["desc_epigrafe"].str.contains("RESTAURANTE")|clean["desc_epigrafe"].str.contains("BAR CON COCINA")]
  
plis = plis.reset_index(drop=True)
plis



,rotulo,desc_epigrafe,desc_seccion,desc_division,id_division,desc_distrito_local,desc_barrio_local,id_barrio_local,cod_barrio_local,desc_situacion_local,coordenada_x_local,coordenada_y_local,id_local
0,BUN & BONES,BAR CON COCINA,HOSTELERÍA,SERVICIOS DE COMIDAS Y BEBIDAS,56,CHAMBERI,GAZTAMBIDE,701,1,Abierto,439509.60,4476421.5,270378224
1,SIN NOTIFICAR,BAR CON COCINA,HOSTELERÍA,SERVICIOS DE COMIDAS Y BEBIDAS,56,CHAMBERI,GAZTAMBIDE,701,1,Abierto,439202.60,4475916.5,270520864
2,ROTULO NO INFORMADO,BAR RESTAURANTE,HOSTELERÍA,SERVICIOS DE COMIDAS Y BEBIDAS,56,CHAMBERI,ALMAGRO,704,4,Abierto,441400.62,4476429.5,270575795
3,MUESTRA,BAR RESTAURANTE,HOSTELERÍA,SERVICIOS DE COMIDAS Y BEBIDAS,56,CHAMBERI,RIOS ROSAS,705,5,Abierto,440958.72,4477470.5,270575832
4,ROTULO NO INFORMADO,BAR RESTAURANTE,HOSTELERÍA,SERVICIOS DE COMIDAS Y BEBIDAS,56,CHAMBERI,ALMAGRO,704,4,Abierto,441072.03,4475416.0,270576667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1640,RUNNING SUSHI,RESTAURANTE,HOSTELERÍA,SERVICIOS DE COMIDAS Y BEBIDAS,56,SALAMANCA,GOYA,402,2,Abierto,442855.60,4475247.5,270538954
1641,RESTAURANTE CHIFA PERUANO C. CAPON,RESTAURANTE,HOSTELERÍA,SERVICIOS DE COMIDAS Y BEBIDAS,56,SALAMANCA,GUINDALERA,404,4,Abierto,442863.60,4476594.5,270527901
1642,CAFÉ SAIGON,RESTAURANTE,HOSTELERÍA,SERVICIOS DE COMIDAS Y BEBIDAS,56,SALAMANCA,CASTELLANA,406,6,Abierto,442054.60,4476196.5,270535195
1643,BIOTZA BAR-RESTAURANTE,BAR RESTAURANTE,HOSTELERÍA,SERVICIOS DE COMIDAS Y BEBIDAS,56,SALAMANCA,RECOLETOS,401,1,Abierto,441728.60,4474998.5,270486171


In [7]:
def hourglass(x, y):
    lat, lon = utm.to_latlon(x, y, 30, 'N')
    return pd.Series({'latitud': lat, 'longitud': lon})

plis[['latitud', 'longitud']] = plis.apply(lambda row: hourglass(row['coordenada_x_local'], row['coordenada_y_local']), axis=1)

plis

plis.to_csv(f"censo_ac.csv", index=False)


In [8]:
map_madrid = folium.Map(location=[40.4168, -3.7038], zoom_start=14)

for (index, row) in plis.iterrows():
    folium.Marker(location = [row.loc["latitud"], row.loc["longitud"]],
    popup = row.loc["rotulo"] + " " + row["desc_distrito_local"],
    tooltip = "click").add_to(map_madrid)

map_madrid
